In [3]:
# Let's start with necessary imports
import os
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization, Input



In [ ]:
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)


SOLUTION GIVEN

In [ ]:
class AnomalyDetector(tf.keras.Model):
    def __init__(self, model_paths, **kwargs):
        super(AnomalyDetector, self).__init__(**kwargs)
        self.model_paths = model_paths
        self.models = [tf.keras.models.load_model(path) for path in model_paths]
        self.weight_layer = tf.keras.layers.Dense(1, activation='sigmoid', use_bias=False)

    def compute_reconstruction_errors(self, input_data):
        reconstruction_errors = []
        for model in self.models:
            reconstructed_data = model(input_data, training=False)
            error = tf.reduce_mean(tf.square(input_data - reconstructed_data), axis=[1, 2])
            reconstruction_errors.append(error)
        return tf.stack(reconstruction_errors, axis=1)

    def call(self, input_data, training=False):
        reconstruction_errors = self.compute_reconstruction_errors(input_data)
        weighted_errors = self.weight_layer(reconstruction_errors)
        return tf.squeeze(weighted_errors, axis=-1)

    def get_config(self):
        # Retrieve the parent's config.
        config = super(AnomalyDetector, self).get_config()
        # Update it with any custom parameters needed to reinstantiate this model.
        config.update({
            "model_paths": self.model_paths,
        })
        return config

    @classmethod
    def from_config(cls, config):
        model_paths = config.pop("model_paths")
        return cls(model_paths, **config)

    def predict_anomalies(self, input_data, threshold=0.5):
        anomaly_scores = self(input_data, training=False)
        is_anomaly = anomaly_scores > threshold
        return is_anomaly, anomaly_scores

In [2]:
base_dir = os.getcwd()  # Use current working directory
model_path = os.path.join(base_dir, 'model.keras')
print(model_path)
class Model:
    def __init__(self):
        self.clf = None
    def predict(self, X, batch_size=1000):
        predictions_batches = []
        for i in range(0, len(X), batch_size):
            batch = X[i:i + batch_size]
            batch_preds = self.clf.predict(batch)
            predictions_batches.append(batch_preds)
        predictions = np.concatenate(predictions_batches, axis=0)
        return predictions.flatten().tolist()

    def load(self):
        print(model_path)
        self.clf = tf.keras.models.load_model(
            model_path,
            custom_objects={'AnomalyDetector': AnomalyDetector}
        )

/Users/gaohang/ligo-anomaly-detection/Submission_file/model.keras
